# 1. 학습 방식 선택
### 작은 모델(예: 1B~3B 파라미터)을 학습시킬 때
#### - Pre-training: 아무것도 모르는 모델에게 대량의 텍스트를 읽혀 언어 구조를 배우게 함.
#### - Fine-tuning: 이미 학습된 모델(예: EXAONE-1.2B, TinyLlama)에 특정 도메인 데이터를 추가로 학습시킴.

# 2. 환경 준비 및 데이터셋 구성
#### - 학습을 위해 모델이 읽을 수 있는 형태로 데이터를 토큰화(Tokenization)

In [1]:
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling

# 1. 예시 데이터셋 생성 TODO .txt나 .json 파일을 로드하는 방식으로 수정 필요
data = [
    {"text": "질문: 사과의 색깔은 무엇인가요? 답변: 사과는 보통 빨간색이나 초록색입니다."},
    {"text": "질문: 하늘은 왜 파란가요? 답변: 빛의 산란 현상 때문입니다."},
]
# # 텍스트 데이터를 토큰화 by Dataset
dataset = Dataset.from_list(data)

# 2. 모델 및 토크나이저 로드 (작은 모델: TinyLlama)
model_id = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token # 패딩 토큰 설정

# 3. 데이터 전처리 함수
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

/opt/anaconda3/envs/data_analysis/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Map: 100%|██████████| 2/2 [00:00<00:00, 45.35 examples/s]


# 3. 좆밥 GPU 위한 효율적인 학습 기법 (LoRA)
#### - GPU 메모리(예: 11GB 내외의 RTX 2080 Ti)가 부족하여, 모델 전체를 학습시키는 대신 일부 파라미터만 학습시키는 LoRA(Low-Rank Adaptation) 방식 활용

#### 1) (가장 중요) 가중치 정밀도 및 로드 설정 (Quantization)
###### 모델 자체의 무게를 줄여 GPU의 기본 점유 공간을 확보하는 것이 가장 중요
##### - load_in_4bit=True (QLoRA): 모델을 4비트로 양자화하여 로드. 16비트 대비 메모리 사용량 약 1/4로 줄임. 20280ti 11GB 메모리라면 7B 모델도 간신히 올릴 수 있을까?
###### - bnb_4bit_compute_dtype=torch.bfloat16: 연산 시의 정밀도 설정. float16보다 bfloat16이 학습 안정성이 높지만, 구형 GPU(2080 Ti 등)는 float16을 사용해야 그나마 속도 나옴.
###### - bnb_4bit_quant_type="nf4": 일반적인 4비트보다 더 정밀한 NormalFloat4 방식을 사용하여 양자화로 인한 성능 저하를 최소화

#### 2) LoRA 하이퍼파라미터 (PEFT Config)
###### 학습 대상이 되는 파라미터의 수 결정.
###### - r (Rank): LoRA 행렬의 크기. 값이 클수록 표현력은 좋아지지만 메모리 사용량 커짐. 보통 8 또는 16을 권장, 메모리가 극도로 부족하면 4까지 도전!!!
###### - target_modules: 학습할 레이어를 지정. ["q_proj", "v_proj"]만 지정하면 메모리를 아낄 수 있고, ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]처럼 모두 지정하면 성능은 좋아지지만 메모리 부하 큼.
###### - task_type="CAUSAL_LM": 텍스트 생성 모델의 경우 이 설정을 명시하여 불필요한 레이어가 학습되는 것을 방지.

#### 3) 배치 사이즈 및 그래디언트 설정 (Training Arguments)
###### 학습 데이터가 GPU를 거칠 때 발생하는 부하를 조절.
###### - per_device_train_batch_size=1: 실제 GPU에 한 번에 올리는 데이터 개수. !!! OOM이 발생하면 무조건 1로 설정 !!!
###### - gradient_accumulation_steps: 부족한 배치 사이즈를 보완하는 핵심 변수. 예를 들어 배치 1로 설정하고 이 값을 16으로 주면, 16번의 계산 결과를 모아서 한 번 가중치를 업데이트. 실제 배치 사이즈 16의 효과를 내면서 메모리는 1만큼만 사용(혜자닷!).
##### - gradient_checkpointing=True: 역전파(Backpropagation) 과정에서 필요한 중간 계산값을 모두 저장하지 않고, 필요할 때 다시 계산. 연산 속도는 약 30% 느려지지만 메모리 점유율을 획기적으로 낮춤(약간 대출 느낌?).

#### 4) 시퀀스 길이 조절 (Data Processing)
###### 데이터 한 건의 길이가 길수록 메모리 사용량은 기하급수적으로 늘어남.
##### - max_seq_length (또는 max_length): 모델이 한 번에 처리하는 토큰 수. 기본 2048, 4096, 메모리가 부족하면 512나 1024로.
###### - group_by_length=True: 길이가 비슷한 데이터끼리 묶어서 배치 학습을 진행. 패딩(Padding) 토큰 낭비를 줄여 효율을 높임.

#### 5) 최적화 도구 (Optimizer & Offloading)
###### 학습 시 사용되는 수학적 최적화 도구의 메모리 점유를 줄임.
##### - optim="paged_adamw_8bit" 또는 "paged_adamw_32bit": 최적화 도구의 상태값을 8비트로 압축하거나, 필요 없는 데이터를 CPU로 잠시 넘기는(Paged) 기능을 활성화. 일반 AdamW보다 메모리를 훨씬 적게 사용.
###### - fp16=True: 16비트 혼합 정밀도 학습을 사용하여 32비트 대비 메모리 사용량을 절반으로.

In [2]:
from peft import LoraConfig, get_peft_model

# LoRA 설정
lora_config = LoraConfig(
    r=8,  # # r(Rank): LoRA 행렬의 크기. 값이 클수록 표현력은 좋아지지만 메모리 사용량이 늘어남. 8 또는 16을 권장, 메모리가 극도로 부족하면 4까지 낮출 수 있습니다.
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"], # 학습할 레이어 지정
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# 모델에 LoRA 적용
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")
model = get_peft_model(model, lora_config)
model.print_trainable_parameters() # 학습 가능한 파라미터 비중 확인

`torch_dtype` is deprecated! Use `dtype` instead!
Some parameters are on the meta device because they were offloaded to the disk.
/opt/anaconda3/envs/data_analysis/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'
trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.1023


# 4. 학습 실행 (Trainer API)
### Hugging Face의 Trainer를 사용

In [ ]:
# 4. 학습 인자 설정
training_args = TrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=4,   # 메모리 상황에 따라 조절
    gradient_accumulation_steps=4,  # 실제 배치 사이즈 = 4 * 4 = 16
    num_train_epochs=3,             # 전체 데이터 반복 횟수
    learning_rate=2e-4,             # 학습률
    fp16=True,                      # 16비트 혼합 정밀도 학습 (속도 향상)
    logging_steps=10,
    save_strategy="epoch",
    optim="paged_adamw_32bit"       # 메모리 효율적인 옵티마이저
)

# 5. 트레이너 초기화 및 학습 시작
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

trainer.train()

# 6. 학습된 모델 저장
model.save_pretrained("./output/my_llm")
tokenizer.save_pretrained("./output/my_tokenizer")